In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
## Preview documents
SELECT * FROM DIRECTORY('@DASH_CORTEX_AGENTS.DATA.DOCS');

In [ ]:
CREATE OR REPLACE TEMPORARY TABLE RAW_TEXT AS
SELECT RELATIVE_PATH,TO_VARCHAR(AI_PARSE_DOCUMENT(to_file(file_url), {'mode': 'layout'}):content) AS EXTRACTED_LAYOUT 
    FROM DIRECTORY('@DASH_CORTEX_AGENTS.DATA.DOCS') 
    WHERE RELATIVE_PATH LIKE '%.pdf'

In [ ]:
select * from RAW_TEXT limit 5;

In [ ]:
create or replace TABLE DASH_CORTEX_AGENTS.DATA.DOCS_CHUNKS_TABLE ( 
    RELATIVE_PATH VARCHAR(16777216), -- Relative path to the PDF file
    CHUNK VARCHAR(16777216), -- Piece of text
    CHUNK_INDEX INTEGER, -- Index for the text
    CATEGORY VARCHAR(16777216) -- Will hold the document category to enable filtering
);


SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER(...)
This Cortex function splits long text into smaller chunks:

'markdown': format of the text
151: max chunk size
25: overlap between chunks
['\n\n', '\n', ' ', '']: preferred split points (paragraphs, lines, spaces, fallback)

Use Case
This is typically used in document intelligence workflows where:

PDFs are parsed into text
Text is chunked for embedding, semantic search, or LLM-based analysis

In [ ]:
insert into DOCS_CHUNKS_TABLE (relative_path, chunk, chunk_index)
    select relative_path, 
            c.value::TEXT as chunk,
            c.INDEX::INTEGER as chunk_index
    from 
        raw_text,
        LATERAL FLATTEN( input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
              EXTRACTED_LAYOUT,
              'markdown',
              152,
              25,
              ['\n\n', '\n', ' ', '']
           )) c;

In [ ]:
SELECT * FROM DOCS_CHUNKS_TABLE limit 7;


In [ ]:
select distinct relative_path from DOCS_CHUNKS_TABLE

In [ ]:
CREATE OR REPLACE TEMPORARY TABLE docs_categories AS WITH unique_documents AS (
  SELECT
    DISTINCT relative_path, chunk
  FROM
    docs_chunks_table
  WHERE 
    chunk_index = 0
  ),
 docs_category_cte AS (
  SELECT
    relative_path,
     AI_CLASSIFY(chunk, ['Electronics', 'Apparel', 'Grocery', 'Beauty Products', 'Home Goods', 'Sport Equipment', 'Store FAQ', 'Summit tent', 'Carver Skis', 'Hydropro Water Bottle', 'Aerox road bike', 'Trailblazer Hiking Boots']):labels[0] AS category
  FROM
    unique_documents
)
SELECT
  *
FROM
  docs_category_cte;

In [ ]:
select * from docs_categories;

In [ ]:
update docs_chunks_table 
  SET category = docs_categories.category
  from docs_categories
  where  docs_chunks_table.relative_path = docs_categories.relative_path;


In [ ]:
insert into DOCS_CHUNKS_TABLE (relative_path, chunk, chunk_index, category)
SELECT 
    RELATIVE_PATH,
    CONCAT('This is a picture describing the bike or ski: '|| RELATIVE_PATH || 
        ' | Description: ' ||
        AI_COMPLETE('claude-4-sonnet',
        'Describe this image: ',
        TO_FILE('@DOCS', RELATIVE_PATH))) as chunk,
    0,
    AI_CLASSIFY(
        TO_FILE('@DOCS', RELATIVE_PATH), ['Electronics', 'Apparel', 'Grocery', 'Beauty Products', 'Home Goods', 'Sport Equipment', 'Store FAQ', 'Summit tent', 'Carver Skis', 'Hydropro Water Bottle', 'Aerox road bike', 'Trailblazer Hiking Boots']):labels[0] as category,
FROM 
    DIRECTORY('@DASH_CORTEX_AGENTS.DATA.DOCS')
WHERE
    RELATIVE_PATH LIKE '%.jpeg';


In [ ]:
select * from DOCS_CHUNKS_TABLE
    where RELATIVE_PATH LIKE '%.jpeg';

In [ ]:
create or replace CORTEX SEARCH SERVICE DASH_CORTEX_AGENTS.DATA.DOCS_SEARCH
ON chunk
ATTRIBUTES relative_path, category
warehouse = AI_WH
TARGET_LAG = '1 hour'
EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
as (
    select chunk,
        chunk_index,
        relative_path,
        category
    from docs_chunks_table
);

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE DASH_CORTEX_AGENTS.DATA.ARTICLE_NAME_SEARCH
  ON ARTICLE_NAME
  WAREHOUSE = AI_WH
  TARGET_LAG = '1 hour'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
AS (
  SELECT
      DISTINCT ARTICLE_NAME
  FROM DASH_CORTEX_AGENTS.DATA.ARTICLE_DIMENSION
);